In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from datasets import load_dataset
from transformer_lens import HookedTransformer

import sae_bench.sae_bench_utils.dataset_utils as dataset_utils

dataset_name = "monology/pile-uncopyrighted"


def get_dataset_list_of_strs(
    dataset_name: str, column_name: str, min_row_chars: int, total_chars: int
) -> list[str]:
    dataset = load_dataset(dataset_name, split="train", streaming=True)

    total_chars_so_far = 0
    result = []

    for row in dataset:
        if len(row[column_name]) > min_row_chars:
            result.append(row[column_name])
            total_chars_so_far += len(row[column_name])
            if total_chars_so_far > total_chars:
                break

    return result


model = HookedTransformer.from_pretrained_no_processing("pythia-70m-deduped")
dataset_list_of_strs = get_dataset_list_of_strs(dataset_name, "text", 100, 10_000_000)

In [ ]:
tokens = dataset_utils.tokenize_and_concat_dataset(
    model.tokenizer,
    dataset_list_of_strs,
    seq_len=128,
    add_bos=True,
    max_tokens=2_000_000,
)

In [ ]:
print(tokens.shape, (tokens.shape[0] * tokens.shape[1]) / 1_000_000, "M tokens")

In [ ]:
tokens2 = dataset_utils.load_and_tokenize_dataset(
    dataset_name, 128, 2_000_000, model.tokenizer
)

In [ ]:
import sae_bench.sae_bench_utils.dataset_info as dataset_info
import sae_bench.sae_bench_utils.dataset_utils as dataset_utils

dataset_name = "canrager/amazon_reviews_mcauley_1and5_sentiment"

In [ ]:
train_data, test_data = dataset_utils.get_multi_label_train_test_data(
    dataset_name, 4000, 1000, 42
)

In [ ]:
print(train_data.keys())
print(train_data["1.0"][0])
print(train_data["5.0"][0])
print(len(train_data["1.0"]))

In [ ]:
new_dataset_name = "codeparrot/github-code"
languages = ["C", "Python", "HTML", "Java", "PHP"]

train_code_data, test_code_data = dataset_utils.get_github_code_dataset(
    new_dataset_name, languages, 4000, 1000, 42
)

In [ ]:
new_dataset_name2 = "fancyzhx/ag_news"
chosen_classes = dataset_info.chosen_classes_per_dataset[new_dataset_name2]

train_news_data, test_news_data = dataset_utils.get_ag_news_dataset(
    new_dataset_name2, chosen_classes, 4000, 1000, 42
)

In [ ]:
new_dataset_name3 = "Helsinki-NLP/europarl"
chosen_classes = dataset_info.chosen_classes_per_dataset[new_dataset_name3]

train_news_data, test_news_data = dataset_utils.get_ag_news_dataset(
    new_dataset_name3, chosen_classes, 4000, 1000, 42
)

In [ ]:
print(train_code_data.keys())
print(train_code_data["C"][0])

In [ ]:
train_data, test_data = dataset_utils.get_multi_label_train_test_data(
    train_df,
    test_df,
    dataset_name,
    4000,
    1000,
    42,
)

chosen_classes = dataset_info.chosen_classes_per_dataset[dataset_name]

train_data = dataset_utils.filter_dataset(train_data, chosen_classes)
test_data = dataset_utils.filter_dataset(test_data, chosen_classes)